# Building a Database for Crime Reports

## Introduction

In this project, I will be applying what I have learnt to build a database for storing data related to crimes that occured in Boston.  The dataset which will be used is stored in a file called `boston.csv`.

The first four rows of the database are shown below.

![image](https://github.com/suzieamey86/Repositaurus/blob/images/Boston%20Crime%20Data.png?raw=true)

A description of the columns is as follows:

- `incident_number`:  An identifier of the crime
- `offense_code`:  A numeric identifier code for the committed crime
- `description`:  A description of the crime
- `date`:  The date on which the crime took place
- `day_of_the_week`:  The day of the week on which the crime took place
- `lat`:  The latitude coordinate at which the crime occurred
- `long`:  The longitude coordinate at which the crime occurred

The goal of the project is to create a database named `crimes_db`, with a table - `boston_crimes` - with appropriate datatypes for storing data from the `boston.csv` file.  The table will be created inside a schema named `crimes`.  I will also create two groups: `readonly` and `readwrite`, with the appropriate privileges assigned to each one.  Finally, one user for each group will be created.

The diagram below shows a high level overview of what will be achieved.

![image](https://github.com/suzieamey86/Repositaurus/blob/images/Crime%20Database%20Overview.png?raw=true)

## Creating the Crime Database

I will start by creating a database named `crime_db` to store the crime data, and a shema named `crimes` to keep the data organised in tables.

In order to create a database, the connection needs to be set to:

- `connection.autocommit = True`

This is because the creation of a database must be executed straightaway by itself, rather than in a transaction block.

In [1]:
import psycopg2

### connect to db 'dq' and create new db ###

conn = psycopg2.connect(dbname = "dq", user = "dq")
cur = conn.cursor()
conn.autocommit = True
cur.execute("CREATE DATABASE crime_db;")

### set autocommit to False after creating new db ###

conn.autocommit = False 
conn.close()

In [12]:
### connect to `crime_db` and create new db schema ###

conn = psycopg2.connect(dbname = "crime_db", user = "dq")
conn.rollback()
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

## Obtaining the Column Names and Sample

Now I can start creating tables.  First, I will look at a sample of the data so that I can decide on which datatypes to use.

I will read the header row and the first data row from the `boston.csv` file and inspect the data.

In [13]:
import csv

### open the file using csv reader and assign a header row ###

with open("boston.csv", "r") as f:
    reader = csv.reader(f)
    rows = list(reader)
    col_headers = rows[0]
    first_row = rows[1]

print(col_headers)
print('\n')
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating an Auxiliary Function

Before creating a table to store the crime data, I need to identify the correct datatypes for each column.  In order to do that, I'm going to create a function named `get_col_value_set()`, which will return a Python set with all distinct values in a column when given the name of a CSV file and the column's index.

This function will be useful to:

- Check whether a column can be represented using an enumerated datatype.
- Calculate the maximum length of any text-like column in order to set an appropriate size for a `VARCHAR` column.

In [14]:
### create function to find distinct values in a given column ###

def get_col_value_set(csv_filename, col_index):
    import csv
    col_values = set()
    with open(csv_filename, "r") as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            col_values.add(row[col_index])
    return col_values

for i in range(len(col_headers)):
    col_values = get_col_value_set("boston.csv", i)
    print(col_headers[i], ": ", len(col_values))

incident_number :  298329
offense_code :  219
description :  239
date :  1177
day_of_the_week :  7
lat :  18177
long :  18177


## Finding the Maximum Length

Now we've got the number of distinct values for each column, we can see that `day_of_the_week` has only got 7 distinct values so would be a good candidate for an enumerated datatype, where the datatype contains only these distinct values, preventing any other value from being entered into the column.

For columns with textual data, it is important to find out the length of the longest word they contain so that we can set a maximum `VARCHAR` length.  The two columns with textual data are `day_of_the_week` and `description`. As I have decided to use an enumerated datatype for the `day_of_the_week` column, I only need to find the longest word in the `description` column, I will use the `get_col_value_set` function to get a set of distinct values in the `description` column (index[2]) and iterate over each element to find out which word is the longest.

In [15]:
### find maximum word length in `description` column ###

description_vals = get_col_value_set("boston.csv", 2)
max_len = 0
for val in description_vals:
    if len(val) > max_len:
        max_len = len(val)
        
print("description max length: ", max_len)

description max length:  58


## Creating the Table

I will now create a table named `boston_crimes` inside the `crimes` schema of the `crime_db` database.  As I have decided to use an enumerated datatype for the `day_of_the_week` column, I will do this first.

In [16]:
### create enumerated datatype for `day_of_the_week` ###

cur.execute("""CREATE TYPE days_of_week AS ENUM
                ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
                    'Saturday', 'Sunday');""")

I will print the values of `col_headers` to remind myself of the information in each column and choose suitable column names for the table, and also print the first 5 data rows, and the last data row, to see what kind of data is contained in each column.

In [17]:
print(col_headers)
print('\n')
for row in rows[1:6]:
    print(row)
    print('\n')
print(rows[-1])

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


['2', '1402', 'VANDALISM', '2018-08-21', 'Tuesday', '42.30682138', '-71.06030035']


['3', '3410', 'TOWED MOTOR VEHICLE', '2018-09-03', 'Monday', '42.34658879', '-71.07242943']


['4', '3114', 'INVESTIGATE PROPERTY', '2018-09-03', 'Monday', '42.33418175', '-71.07866441']


['5', '3114', 'INVESTIGATE PROPERTY', '2018-09-03', 'Monday', '42.27536542', '-71.09036101']


['298329', '3125', 'WARRANT ARREST', '2015-06-22', 'Monday', '42.33383935', '-71.08029038']


The names of the columns in the new table and their datatypes will be:

- Incident_Number: integer
- Offense_Code: smallint
- Description_of_Crime: varchar(100)
- Date_of_Crime: date
- Day_of_Crime: Enumerated (`days_of_week`)
- Latitude: decimal (10, 8)
- Longitude: decimal (11, 8)

My reasons for datatype selection are:

- Incident_Number: This column contains a number identifying the crime, it starts from the number 1 and, I can see from the last row of data printed above, that it goes up to the number 298329.  As the integer datatype has a range of -2147483648 to +2147483647, it will be able to accomodate the highest number in this column.
- Offense_Code: The numbers in this column look to be either 3 or 4 digits long, easily accomodated by the smallint range of -32768 to +32767.
- Description of Crime: We calculated previously that the longest entry in this column is 58 characters long.  I have given the varchar a maximum value of 100 to allow some margin but still keep the memory requirement fairly small.
- Date_of_Crime: This is fairly self-explanatory. A standard date format can be used.
- Day_of_Crime: I have created the enumerated datatype `days_of_week` for this column.
- Latitude: The highest possible value for a latitude coordinate is -90 or +90, so there will be no more than two digits before the decimal place.  The values I can see from the data sample show an accuracy of 8 digits after the decimal place.
- Longitude: The higest possible value for a longitude coordinate is -180 or +180, so I have given this decimal datatype a precision of 11, and again, a score of 8 digits after the decimal place.

Now I can go ahead and create the table with the correct datatype for each column.

In [18]:
### create `boston_crimes` table within the `crimes` schema ###

cur.execute("""
        CREATE TABLE crimes.boston_crimes (
            Incident_Number integer PRIMARY KEY,
            Offense_Code smallint,
            Description_of_Crime varchar(100),
            Date_of_Crime date,
            Day_of_Crime days_of_week,
            Latitude decimal(10, 8),
            Longitude decimal(11, 8)
        );
    """)

## Loading the Data

Now that the `boston_crimes` table has been created, I can load the data from the `boston.csv` file into it. 

I am going to use the `cursor.copy_expert()` method to do this.

In [19]:
### load data into `boston_crimes` table from `boston.csv` ###

with open("boston.csv", "r") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

### print the no. of table rows to check data loaded correctly ###

cur.execute("SELECT * FROM crimes.boston_crimes")
print(len(cur.fetchall()))

298329


We have now got 298329 rows of data in the `crimes.boston_crimes` table, as expected.

## Revoking Public Privileges

Before creating my two user groups, namely `readonly` and `readwrite`, I want to ensure that no privileges can be inherited from the `public` group and `public` schema to any other group or schema, so I will revoke all privileges on the `public` group and schema.  This practice follows the privilege of least principle, where any user or group should only have the least amount of access necessary for their role.

In [20]:
### revoke all privileges from `public` group and schema ###

cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating User Groups

Now I can create the two user groups; these are `readonly`, where users will only be able to read data, so will only have privileges to perform `SELECT` queries, and `readwrite`, where users will have privileges to perform `SELECT`, `INSERT`, `DELETE` and `UPDATE` queries, so, they can read and alter data but not delete any tables.

I will create the user groups with `NOLOGIN` because log-in details should be given to individual users rather than groups.

Both user groups will also need connection privileges to the `crime_db` database, otherwise they won't be able to connect or do anything at all with this database, and they will need to be granted usage of the `crimes` schema within the database.

In [21]:
### create `readonly` and `readwrite` user groups ###

cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")

### grant `CONNECT` to crime_db and USAGE to `crimes` schema for both groups ###

cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

### grant group specific privileges to each group for all tables in `crimes` schema ###

cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

## Creating Users 

Next, I will create one user for each of the new user groups with individual user names and passwords:

- readonly: user = "data_analyst", password = "secret1"
- readwrite: user = "data_scientist", password = "secret2"

Each user also needs to be granted the privileges for the correct user group.

In [22]:
### create a user for each new user group ###

cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

## Testing

The database is now set up with a schema, a table, two user groups and a user for each user group.  It is good practice to test that everything is configured as expected once the set-up is complete.  To do this, we can query Postgres internal tables to check that objects have been created and users and groups have been assigned the correct privileges.

I will query the `pg_roles` table to inspect the privileges related to the database, the columns I will select are:

- rolname: the role name
- rolsuper: states whether the role has superuser privileges
- rolcreaterole: whether the group can create other roles
- rolcreatedb: whether the group can create databases 
- rolcanlogin: whether the user/group can log in

`rolsuper`, `rolcreaterole`, and `rolcreatedb` are expected to be False for both groups and both users which have been created, and `rolcanlogin` is expected to be False for the two groups but True for the two users.

To inspect the table privileges, I will query the following columns in the `information_schema.table_privileges` table:

- grantee: name of the role that the privilege was granted to
- privilege_type: type of the privilege that was granted

In [24]:
### check database privileges for users and user groups ###

cur.execute("""
    SELECT
        rolname,
        rolsuper,
        rolcreaterole,
        rolcreatedb,
        rolcanlogin
    FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');""")

db_roles = cur.fetchall()
for row in db_roles:
    print(row)
print('\n')
    
### check table privileges for user groups ###

cur.execute("""
    SELECT
        grantee,
        privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');""")

privileges = cur.fetchall()
for row in privileges:
    print(row)

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)


('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')


As expected, both of the user groups returned False for all the selected `pg_roles` columns, and both users returned False for all columns except `rolcanlogin`.

Also, we can see that the correct privileges have been assigned to both user groups.

## Conclusion

After testing the database, I am happy that all of the data from the `boston.csv` file has been entered, the two user groups with correct privileges have been created, and these privileges assigned to the correct users.